In [18]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os 
import time
import pandas as pd
import numpy as np
import re


In [19]:
os.getcwd()
os.chdir("C:/Users/hakos/GDrive/Downloads/Research/data/yakka")
data=pd.read_csv('../merged/len3_ndb_blp_DN_firm_val.csv',encoding='utf-8',index_col=0)
data.columns

C:\Users\hakos\AppData\Local\Temp\ipykernel_28924\388202102.py:3: DtypeWarning: Columns (118,119,131) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv('../merged/len3_ndb_blp_DN_firm_val.csv',encoding='utf-8',index_col=0)


Index(['薬効分類', '薬効分類名称', '医薬品コード', '医薬品名', '薬価基準収載医薬品コード', '薬価_x', '後発品区分',
       '総計', 'year', 'in_hospital',
       ...
       '同一剤形・規格の後発医薬品がある先発医薬品', '薬価_y', '経過措置による使用期限', '備考', 'Unnamed: 15',
       'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 4', '_merge'],
      dtype='object', length=132)

In [20]:
len(set(data[data["メーカー名"].isna()]["医薬品名"]))
data[data["メーカー名"].isna()]["医薬品名"].value_counts()
med_name=list(set(data[data["メーカー名"].isna()]["医薬品名"]))
len(med_name)

112

In [21]:
# import re
# import unicodedata
# name =   'プロカテロール塩酸塩シロップ５μｇ／ｍＬ\u3000日医工\u3000０．０００５％'

# # 全角スペース（\u3000）を正規表現で削除
# match = re.search(r'.*\u3000', name)

# if match:
#     result = match.group()
# else:
#     result = name
# result = re.sub(r'\u3000', ' ', result)
# result = unicodedata.normalize('NFKC', result)
# print(result)
# results = [
#     unicodedata.normalize('NFKC', re.sub(r'\u3000', ' ', re.search(r'.*\u3000', name).group()) if re.search(r'.*\u3000', name) else name)
#     for name in med_name
# ]

# # 結果を出力
# for result in results:
#     print(result)

In [22]:
import re
import unicodedata


# クリーン前のテキストをキー、クリーン後のテキストを値とする辞書を作成
name2cleaned = {}
cleaned=[]
# テキストごとに処理を行うループ
for name in med_name:
    # クリーン処理を実行
    cleaned_name = unicodedata.normalize('NFKC', re.sub(r'\u3000', ' ', re.search(r'.*\u3000', name).group()) if re.search(r'.*\u3000', name) else name)
    
    cleaned.append(cleaned_name)
    # クリーン前のテキストをキー、クリーン後のテキストを値として辞書に追加
    name2cleaned[name] = cleaned_name

# 辞書を出力
# for original_name in name2cleaned.keys():
#     print(f'Original name: {original_name}')
#     print(f'Cleaned name:  {name2cleaned[original_name]}')
#     print()


In [23]:
#　初期化　危ない
clean=0
name2firm={}
name2name={}

In [24]:
def get_maker(key):
    # Geckodriverのパスを指定してインスタンスを作成
    os.chdir("C:\\Users\\hakos\\Downloads")
    os.getcwd()
    driver = webdriver.Firefox()
    driver.minimize_window()
    # ウェブページを開く
    driver.get('https://www.pmda.go.jp/PmdaSearch/iyakuSearch/')

    wait = WebDriverWait(driver, 5)
    search_box = wait.until(EC.presence_of_element_located((By.ID, 'txtName')))
    search_box.clear()
    search_box.send_keys(key)
    search_box.send_keys(Keys.RETURN)

    # search_button = driver.find_element(By.XPATH, '//input[@name="btnA"]')  # XPathを使用してボタンを指定
    # search_button.click()

    # 結果を取得

    time.sleep(2)

    # 自分で遷移する必要がある
    driver.switch_to.window(driver.window_handles[1])
    try:
        element = driver.find_element(By.XPATH, "//tr[@class='TrColor01']//div[contains(text(), '製造販売元')]")
        maker = element.text
        maker= maker.replace("製造販売元／", "").replace("株式会社", "")
        try:
            element=driver.find_element(By.XPATH, "//tr[@class='TrColor01']/td[2]/div")
            name=element.text
        except:
            name=None
            # print("name error")
    except:
        maker = None
        name=None
    # ページが完全に読み込まれるのを待つ（任意）
    time.sleep(2)
    # ブラウザを閉じる
    driver.quit()
    return (maker,name)
none_count=0
# med_name=list(set(data[data["メーカー名"].isna()]["医薬品名"]))
# test
# get_maker("アレグラ")


In [26]:
failed=[]
if clean:
    med_name=cleaned
for i,key in enumerate(med_name):
    if key not in name2firm.keys():
        value,name=get_maker(key)
        if value is None:
            none_count+=1
            failed.append(key)
        else:
            name2firm[key]=value
            name2name[key]=name
            # print(i,key,value,name)
    else:
        continue
    print(i,end=" ")
    time.sleep(3)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 

In [28]:
name2firm, name2name

({'複方ヨード・グリセリン': '小堺製薬',
  'エパルレスタット錠５０ｍｇ「オーハラ」': '大原薬品工業',
  '亜鉛華軟膏': '日興製薬',
  'ビルヘキサル錠２００ｍｇ': ',テイコクメディックス',
  'ジスチグミン臭化物錠\u3000５ｍｇ': '日医工岐阜工場',
  'ビルヘキサル錠４００ｍｇ': ',テイコクメディックス',
  'アシクロビン顆粒４０％': ',テイコクメディックス',
  'アシクロビル錠２００ｍｇ「マイラン」': ',テイコクメディックス',
  'カモスタットメシル酸塩錠１００ｍｇ「オーハラ」': '大原薬品工業',
  '薬用炭': '日医工',
  'イオウ': '小堺製薬',
  '乳酸': 'エイワイファーマ',
  'Ｄソルビトール': '大塚製薬工場'},
 {'複方ヨード・グリセリン': '複方ヨード・グリセリン「コザカイ・Ｍ」',
  'エパルレスタット錠５０ｍｇ「オーハラ」': 'エパルレスタット錠５０ｍｇ「オーハラ」',
  '亜鉛華軟膏': '亜鉛華(10%)単軟膏「ニッコー」',
  'ビルヘキサル錠２００ｍｇ': 'アイラックス錠200mg／アイラックス錠400mg',
  'ジスチグミン臭化物錠\u3000５ｍｇ': 'ジスチグミン臭化物錠5mg「ＮＩＧ」',
  'ビルヘキサル錠４００ｍｇ': 'アイラックス錠200mg／アイラックス錠400mg',
  'アシクロビン顆粒４０％': 'アイラックス錠200mg／アイラックス錠400mg',
  'アシクロビル錠２００ｍｇ「マイラン」': 'アイラックス錠200mg／アイラックス錠400mg',
  'カモスタットメシル酸塩錠１００ｍｇ「オーハラ」': 'カモスタットメシル酸塩錠100mg「オーハラ」',
  '薬用炭': '薬用炭「日医工」',
  'イオウ': 'イオウ「コザカイ・Ｍ」',
  '乳酸': 'ソリタックス‐Ｈ輸液',
  'Ｄソルビトール': 'オーツカＭＶ注'})

In [29]:
import pickle
os.chdir("C:/Users/hakos/GDrive/Downloads/Research/data/yakka")

# Open the file in binary write mode and dump the dictionary into it
# clean=0
# with open('name2firm.pkl', 'rb') as f:
#     name2firm = pickle.load(f)
if clean:
    file_path = './pickle/cleaned2firm.pkl'
    with open(file_path, 'wb') as file:
        pickle.dump(name2firm, file)
    file_path = './pickle/name2cleaned.pkl'
    with open(file_path, 'wb') as file:
        pickle.dump(name2cleaned, file)
else:
    file_path = './pickle/name2firm.pkl'
    with open(file_path, 'wb') as file:
        pickle.dump(name2firm, file)
    file_path = './pickle/name2name.pkl'
    with open(file_path, 'wb') as file:
        pickle.dump(name2name, file)


In [32]:
data.loc[data["メーカー名"].isna(),"メーカー名"]=data.loc[data["メーカー名"].isna(),"医薬品名"].map(name2cleaned).map(name2firm)
data.loc[data["メーカー名"].isna(),"メーカー名"]=data.loc[data["メーカー名"].isna(),"医薬品名"].map(name2firm)
data[data["メーカー名"].isna()].shape

(374, 132)

In [33]:
len(set(data[data["メーカー名"].isna()]["医薬品名"]))

99

In [13]:
os.chdir("C:/Users/hakos/GDrive/Downloads/Research/data/yakka")
# data.to_csv('../merged/len3_ndb_blp_DN_firm_filled.csv',encoding='utf-8')